In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/kaggle/input/congestion-data/us_congestion_2016_2022_sample_2m.csv')

In [3]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, accuracy_score

In [4]:
df = df.drop(["Description", "Street", "City", "County", "State", "Country", "ZipCode", "LocalTimeZone",
               "WeatherStation_AirportCode", "WeatherTimeStamp","ID"], axis=1, errors="ignore")

In [5]:
df_original = pd.read_csv('/kaggle/input/congestion-data/us_congestion_2016_2022_sample_2m.csv')

In [6]:
encoding_maps = {}

In [7]:
categorical_cols = ["Weather_Event", "Weather_Conditions", "WindDir","Congestion_Speed"]
for col in categorical_cols:
    encoder = LabelEncoder()
    encoder.fit(df_original[col].astype(str))  # Fit on the original unencoded data
    encoding_maps[col] = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))  # Store mapping

In [8]:
categorical_cols = ["Weather_Event", "Weather_Conditions", "WindDir","Congestion_Speed"]
for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

In [9]:
weather_conditions_map = encoding_maps["Weather_Conditions"]
print(weather_conditions_map)

{'Blowing Dust': 0, 'Blowing Dust / Windy': 1, 'Blowing Sand': 2, 'Blowing Snow': 3, 'Blowing Snow / Windy': 4, 'Blowing Snow Nearby': 5, 'Blowing Snow Nearby / Windy': 6, 'Clear': 7, 'Cloudy': 8, 'Cloudy / Windy': 9, 'Drifting Snow': 10, 'Drizzle': 11, 'Drizzle / Windy': 12, 'Drizzle and Fog': 13, 'Fair': 14, 'Fair / Windy': 15, 'Fog': 16, 'Fog / Windy': 17, 'Freezing Drizzle': 18, 'Freezing Rain': 19, 'Freezing Rain / Windy': 20, 'Funnel Cloud': 21, 'Hail': 22, 'Haze': 23, 'Haze / Windy': 24, 'Heavy Blowing Snow': 25, 'Heavy Drizzle': 26, 'Heavy Freezing Drizzle': 27, 'Heavy Freezing Rain': 28, 'Heavy Ice Pellets': 29, 'Heavy Rain': 30, 'Heavy Rain / Windy': 31, 'Heavy Rain Shower': 32, 'Heavy Rain Showers': 33, 'Heavy Sleet': 34, 'Heavy Sleet / Windy': 35, 'Heavy Sleet and Thunder': 36, 'Heavy Snow': 37, 'Heavy Snow / Windy': 38, 'Heavy Snow Showers': 39, 'Heavy Snow with Thunder': 40, 'Heavy T-Storm': 41, 'Heavy T-Storm / Windy': 42, 'Heavy Thunderstorm': 43, 'Heavy Thunderstorms a

In [10]:
weather_event_map = encoding_maps["Weather_Event"]
print(weather_event_map)

{'Fog': 0, 'Fog ;Rain': 1, 'Fog ;Rain ;Hail ;Thunderstorm': 2, 'Fog ;Rain ;Snow': 3, 'Fog ;Rain ;Snow ;Thunderstorm': 4, 'Fog ;Rain ;Thunderstorm': 5, 'Fog ;Rain ;Thunderstorm ;Tornado': 6, 'Fog ;Snow': 7, 'Fog ;Snow ;Thunderstorm': 8, 'Fog ;Thunderstorm': 9, 'Hail': 10, 'Hail ;Thunderstorm': 11, 'Rain': 12, 'Rain ;Hail ;Thunderstorm': 13, 'Rain ;Snow': 14, 'Rain ;Snow ;Thunderstorm': 15, 'Rain ;Thunderstorm': 16, 'Rain ;Thunderstorm ;Tornado': 17, 'Snow': 18, 'Snow ;Hail': 19, 'Snow ;Thunderstorm': 20, 'Thunderstorm': 21, 'Thunderstorm ;Tornado': 22, 'Tornado': 23, 'nan': 24}


In [11]:
df["StartTime"] = pd.to_datetime(df["StartTime"], utc=True, errors="coerce")
df["EndTime"] = pd.to_datetime(df["EndTime"], utc=True, errors="coerce")

In [12]:
df = df.dropna(subset=["StartTime", "EndTime"])

In [13]:
df["StartTime"] = (df["StartTime"] - df["StartTime"].min()).dt.total_seconds()
df["EndTime"] = (df["EndTime"] - df["EndTime"].min()).dt.total_seconds()

In [14]:
X = df.drop(["DelayFromTypicalTraffic(mins)", "Congestion_Speed"], axis=1)
y_regression = df["DelayFromTypicalTraffic(mins)"]  # Regression target
y_classification = df["Congestion_Speed"]  # Binary Classification (0 = Fast, 1 = Moderate, 2 = Slow)

In [15]:
X_train, X_test, y_reg_train, y_reg_test, y_cls_train, y_cls_test = train_test_split(
    X, y_regression, y_classification, test_size=0.2, random_state=42
)

In [16]:
model = xgb.XGBRegressor(objective="reg:squarederror", enable_categorical=True)
clf = xgb.XGBClassifier(objective="binary:logistic", enable_categorical=True)

In [17]:
model.fit(X_train, y_reg_train)
clf.fit(X_train, y_cls_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [18]:
y_reg_pred = model.predict(X_test)
y_cls_pred = clf.predict(X_test)

In [19]:
mae = mean_absolute_error(y_reg_test, y_reg_pred)
accuracy = accuracy_score(y_cls_test, y_cls_pred)

print(f"Mean Absolute Error (MAE) for Delay Prediction: {mae}")
print(f"Accuracy for Congestion Prediction: {accuracy}")

Mean Absolute Error (MAE) for Delay Prediction: 0.7487718644020829
Accuracy for Congestion Prediction: 0.7246844628612037


In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_cls_test, y_cls_pred))

              precision    recall  f1-score   support

           0       0.80      0.74      0.77    143084
           1       0.67      0.74      0.70    159343
           2       0.71      0.68      0.70     97209

    accuracy                           0.72    399636
   macro avg       0.73      0.72      0.72    399636
weighted avg       0.73      0.72      0.73    399636



In [21]:
importance = model.feature_importances_
for feature, score in zip(X_train.columns, importance):
    print(f"{feature}: {score:.4f}")

Severity: 0.0288
Start_Lat: 0.0275
Start_Lng: 0.0342
StartTime: 0.0393
EndTime: 0.0543
Distance(mi): 0.0496
DelayFromFreeFlowSpeed(mins): 0.5441
Temperature(F): 0.0516
WindChill(F): 0.0228
Humidity(%): 0.0216
Pressure(in): 0.0230
Visibility(mi): 0.0209
WindDir: 0.0156
WindSpeed(mph): 0.0116
Precipitation(in): 0.0163
Weather_Event: 0.0257
Weather_Conditions: 0.0131


In [22]:
import optuna

In [23]:
def objective(trial):
    params = {
        "objective": "reg:squarederror",  # Regression for DelayFromTypicalTraffic
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
    }

    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_reg_train)

    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_reg_test, y_pred)

    return mae

In [24]:
# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=30)

In [25]:
best_params = {'n_estimators': 900, 'max_depth': 12, 'learning_rate': 0.05343119005856983, 'subsample': 0.897285760757781, 'colsample_bytree': 0.9980782272439057, 'reg_alpha': 0.018441364690612083, 'reg_lambda': 0.19361190668945039, 'min_child_weight': 6}
print("Best parameters:", best_params)

Best parameters: {'n_estimators': 900, 'max_depth': 12, 'learning_rate': 0.05343119005856983, 'subsample': 0.897285760757781, 'colsample_bytree': 0.9980782272439057, 'reg_alpha': 0.018441364690612083, 'reg_lambda': 0.19361190668945039, 'min_child_weight': 6}


In [26]:
best_model = xgb.XGBRegressor(**best_params)
best_model.fit(X_train, y_reg_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9980782272439057, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05343119005856983, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=12, max_leaves=None,
             min_child_weight=6, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=900, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [27]:
y_pred_final = best_model.predict(X_test)
mae_final = mean_absolute_error(y_reg_test, y_pred_final)

In [28]:
print(f"Final Mean Absolute Error (MAE): {mae_final}")

Final Mean Absolute Error (MAE): 0.653894843878839


In [29]:
def objective_cls(trial):
    params = {
        "objective": "multi:softmax", 
        "num_class": 3,  
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
    }

    clf = xgb.XGBClassifier(**params)
    clf.fit(X_train, y_cls_train)

    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_cls_test, y_pred)

    return 1 - acc  

In [30]:
# study_cls = optuna.create_study(direction="minimize")
# study_cls.optimize(objective_cls, n_trials=30)

In [31]:
best_params_cls = {'n_estimators': 800, 'max_depth': 10, 'learning_rate': 0.09219203992461676, 'subsample': 0.9440038119184523, 'colsample_bytree': 0.6556610437910383, 'reg_alpha': 0.5921488001440582, 'reg_lambda': 0.27768888744961756, 'min_child_weight': 1}
print("Best parameters for classification:", best_params_cls)

Best parameters for classification: {'n_estimators': 800, 'max_depth': 10, 'learning_rate': 0.09219203992461676, 'subsample': 0.9440038119184523, 'colsample_bytree': 0.6556610437910383, 'reg_alpha': 0.5921488001440582, 'reg_lambda': 0.27768888744961756, 'min_child_weight': 1}


In [32]:
best_clf = xgb.XGBClassifier(**best_params_cls)
best_clf.fit(X_train, y_cls_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6556610437910383, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.09219203992461676,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=800, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [33]:
y_pred_cls = best_clf.predict(X_test)
acc_final = accuracy_score(y_cls_test, y_pred_cls)

In [34]:
print(f"Final Classification Accuracy: {acc_final}")

Final Classification Accuracy: 0.7544390395259686


In [35]:
!pip install transformers accelerate torch sentencepiece --quiet

In [36]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.2 MB/s eta 0:00:00


In [37]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_KEY")

In [38]:
from huggingface_hub import login
login(token=secret_value_0)

In [39]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

In [40]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [41]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_8bit=True,  
    device_map="auto"  
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [42]:
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [43]:
weather_conditions_map_rev = {v: k for k, v in weather_conditions_map.items()} 
weather_event_map_rev = {v: k for k, v in weather_event_map.items()} 

In [44]:
def generate_explanation(features, delay_pred, congestion_pred):
    weather_condition = weather_conditions_map_rev.get(features["Weather_Conditions"], "Unknown")
    weather_event = weather_event_map_rev.get(features["Weather_Event"], "Unknown")

    prompt = f"""
    Given the following traffic data:
    - Weather Conditions: {weather_condition} (Event: {weather_event})
    - Temperature: {features['Temperature(F)']}°F
    - Wind Speed: {features['WindSpeed(mph)']} mph
    - Wind Direction: {features['WindDir']}
    - Visibility: {features['Visibility(mi)']} miles
    - Precipitation: {features['Precipitation(in)']} inches

    The model has predicted:
    - Delay: {delay_pred:.2f} minutes
    - Congestion Level: {"Fast" if congestion_pred == 0 else "Moderate" if congestion_pred == 1 else "Slow"}
    
    Explain why the delay and congestion level might be as predicted. Keep the explanation concise and suitable for traffic management personnel. Suggest measures to handle the predicted situation.
    """

    response = llm_pipeline(prompt, max_new_tokens=200, return_full_text=False)[0]["generated_text"]
    return response

In [45]:
sample_idx = 18
sample_features = df.iloc[sample_idx].to_dict()
sample_delay_pred = y_reg_pred[sample_idx]
sample_congestion_pred = y_cls_pred[sample_idx]

In [46]:
explanation = generate_explanation(sample_features, sample_delay_pred, sample_congestion_pred)
print(explanation)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



    The delay and congestion level might be as predicted due to the combination of snow and fog conditions, which can reduce visibility and make it difficult for drivers to see the road ahead. Additionally, the low temperature and wind speed can also contribute to the delay and congestion.
    
    To handle the predicted situation, traffic management personnel can consider implementing measures such as reducing speed limits, increasing signage and warning systems, and deploying additional traffic control personnel to manage the flow of traffic. Additionally, they can also consider closing certain roads or lanes to reduce congestion and improve safety.
